> CART分类树采用**Gini指数**来进行特征选择
> 完整的CART算法包括特征选择、决策树生成和决策树剪枝三个部分。
![微信图片_20200518122233.jpg](https://user-gold-cdn.xitu.io/2020/5/18/172260521433a2a1?w=1080&h=413&f=jpeg&s=53046)
> CART是在**给定输入随机变量X条件下输出随机变量Y的条件概率分布**的学习方法。CART算法通过选择最优特征和特征值进行划分，将输入空间也就是特征空间划分为有限个单元，并在这些单元上确定预测的概率分布，也就是在输入给定的条件下输出条件概率分布。
>
> CART算法主要包括**回归树和分类树**两种。回归树用于**目标变量为连续型的建模任务，其特征选择准则用的是平方误差最小准则**。分类树用于**目标变量为离散型的的建模任务，其特征选择准则用的是基尼指数(Gini Index)**，这也有别于此前ID3的信息增益准则和C4.5的信息增益比准则。无论是回归树还是分类树，其算法核心都在于**递归地选择最优特征构建决策树**。
>
> 除了选择最优特征构建决策树之外，CART算法还包括另外一个重要的部分：**剪枝**。剪枝可以视为决策树算法的一种**正则化手段**，作为一种基于规则的非参数监督学习方法，决策树在训练很容易过拟合，导致最后生成的决策树泛化性能不高。
> 另外，CART作为一种单模型，也是GBDT的基模型。当很多棵CART分类树或者回归树集成起来的时候，就形成了GBDT模型。
> 
> Gini指数是针对概率分布而言的。假设在一个分类问题中有K个类，样本属于第k个类的概率$P_k$，则该样本概率分布的基尼指数为
<center>$Gini(P)=\sum_{k=1}^{n}{(P_k*(1-P_k)}$ </center>
[参考文章](https://mp.weixin.qq.com/s/jdUQIPM2AhAh7rzl1DPgIQ)

# 获取数据


In [7]:
import pandas as pd 
df=pd.read_excel('sales_data.xls').iloc[:,1:]
df[["天气","销量"]]

,天气,销量
0,坏,高
1,坏,高
2,坏,高
3,坏,高
4,坏,高
5,坏,高
6,坏,高
7,好,高
8,好,高
9,好,高


# 定义Gini指数的计算函数

In [ ]:
def gini(ele):
    prob=[ele.count(i)/len(ele) for i in set(ele)]
    return sum(i*(1-i) for i in prob)
gini(df["天气"].tolist())

# 定义根据特征分割数据框的函数

In [3]:
def split_df(df,label):
    result={}
    for k in df[label].unique():
        result[k]=df[df[label]==k]
    return result
split_df(df,'天气')

{'坏':    天气 周末 促销 销量
 0   坏  是  是  高
 1   坏  是  是  高
 2   坏  是  是  高
 3   坏  否  是  高
 4   坏  是  是  高
 5   坏  否  是  高
 6   坏  是  否  高
 13  坏  是  是  低
 19  坏  否  否  低
 20  坏  否  是  低
 21  坏  否  是  低
 22  坏  否  是  低
 23  坏  否  否  低
 24  坏  是  否  低
 27  坏  否  否  低
 28  坏  否  否  低
 30  坏  是  否  低,
 '好':    天气 周末 促销 销量
 7   好  是  是  高
 8   好  是  否  高
 9   好  是  是  高
 10  好  是  是  高
 11  好  是  是  高
 12  好  是  是  高
 14  好  否  是  高
 15  好  否  是  高
 16  好  否  是  高
 17  好  否  是  高
 18  好  否  否  高
 25  好  否  是  低
 26  好  否  是  低
 29  好  否  否  低
 31  好  否  是  低
 32  好  否  否  低
 33  好  否  否  低}

# 根据Gini指数和条件Gini指数计算递归选择最优特征

In [4]:
def choose_best_col(df,label):
    cols=[c for c in df.columns if c!=label]
    min_gini,min_split_df,best_col=999,None,""
    for col in cols:
        gini_A=.0 
        splited_subset=split_df(df,col)
        for split_v,split_data in splited_subset.items():
            gini_K=gini(split_data[label].tolist())
            gini_A+=(len(split_data)/len(df))*gini_K
        if min_gini>gini_A:
            min_gini,min_split_df,best_col=gini_A,splited_subset,col 
    return min_gini,min_split_df,best_col
choose_best_col(df,"天气")

(0.4722222222222222,
 {'高':    天气 周末 促销 销量
  0   坏  是  是  高
  1   坏  是  是  高
  2   坏  是  是  高
  3   坏  否  是  高
  4   坏  是  是  高
  5   坏  否  是  高
  6   坏  是  否  高
  7   好  是  是  高
  8   好  是  否  高
  9   好  是  是  高
  10  好  是  是  高
  11  好  是  是  高
  12  好  是  是  高
  14  好  否  是  高
  15  好  否  是  高
  16  好  否  是  高
  17  好  否  是  高
  18  好  否  否  高,
  '低':    天气 周末 促销 销量
  13  坏  是  是  低
  19  坏  否  否  低
  20  坏  否  是  低
  21  坏  否  是  低
  22  坏  否  是  低
  23  坏  否  否  低
  24  坏  是  否  低
  25  好  否  是  低
  26  好  否  是  低
  27  坏  否  否  低
  28  坏  否  否  低
  29  好  否  否  低
  30  坏  是  否  低
  31  好  否  是  低
  32  好  否  否  低
  33  好  否  否  低},
 '销量')

# 定义CART分类树的构建过程

In [17]:
class CartTree:
    class Node:
        def __init__(self,name):
            self.name,self.connections=name,{}
        def connect(self,label,node):
            self.connections[label]=node
    def __init__(self,df,label):
        self.df,self.columns,self.label,self.root=df,df.columns,label,self.Node('Root')
    def construct(self,parent_node,parent_con_label,df,columns):
        min_gini,min_split_df,best_col=choose_best_col(df[columns],self.label)
        if not best_col:
            node=self.Node(df[self.label].iloc[0])
            parent_node.connect(parent_con_label,node)
        else:
            node=self.Node(best_col)
            parent_node.connect(parent_con_label,node)
            new_columns=[c for c in columns if c!=best_col]
            for split_v,split_data in min_split_df.items():
                self.construct(node,split_v,split_data,new_columns)
    def construct_tree(self):
        self.construct(self.root,"Root",self.df,self.columns)
    def print_tree(self,root,tabs):
        print(f"{tabs}({root.name})")
        for label ,node in root.connections.items():
            print(f"{tabs}\t| <{label}>")
            self.print_tree(node,tabs+"\t\t|")

tree=CartTree(df,"天气")
tree.construct_tree()
tree.print_tree(tree.root,"| ")


| (Root)
| 	| <Root>
| 		|(销量)
| 		|	| <高>
| 		|		|(周末)
| 		|		|	| <是>
| 		|		|		|(促销)
| 		|		|		|	| <是>
| 		|		|		|		|(坏)
| 		|		|		|	| <否>
| 		|		|		|		|(坏)
| 		|		|	| <否>
| 		|		|		|(促销)
| 		|		|		|	| <是>
| 		|		|		|		|(坏)
| 		|		|		|	| <否>
| 		|		|		|		|(好)
| 		|	| <低>
| 		|		|(周末)
| 		|		|	| <是>
| 		|		|		|(促销)
| 		|		|		|	| <是>
| 		|		|		|		|(坏)
| 		|		|		|	| <否>
| 		|		|		|		|(坏)
| 		|		|	| <否>
| 		|		|		|(促销)
| 		|		|		|	| <否>
| 		|		|		|		|(坏)
| 		|		|		|	| <是>
| 		|		|		|		|(坏)
